In [1]:
library("reticulate")
use_python("C:/Users/berkc/Miniconda3/envs/R", required = T)
py_config()

python:         C:/Users/berkc/Miniconda3/envs/R/python.exe
libpython:      C:/Users/berkc/Miniconda3/envs/R/python36.dll
pythonhome:     C:\Users\berkc\Miniconda3\envs\R
version:        3.6.10 (default, Mar  5 2020, 10:17:47) [MSC v.1900 64 bit (AMD64)]
Architecture:   64bit
numpy:          C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\numpy
numpy_version:  1.17.0

NOTE: Python version was forced by use_python function

In [2]:
source("sobolGP_poro_np.R")
source("sobolGPmethods.R")
source("pickfreeze.R")

In [ ]:
d <- 2
numobs <- c(5,10,20,30,38)
# numobs <- c(20)
n <- 5000
X1 <- (matrix(runif(d * n), nrow = n))
X2 <- (matrix(runif(d * n), nrow = n))

candidate <- data.frame(matrix(runif(d * 2), nrow = 2))

for(i in numobs){
    
    #--------------------------------------#
    # Data
    #--------------------------------------#
    dat <- read.table('C:/Users/berkc/git/Physics-Informed-Machine-Learning/data/labeled_data.dat')
    x_exp <- dat[1:i,1:2]
    y_exp <- dat[1:i,(ncol(dat)-1)]
#     colnames(x_exp)<- c("X1","X2")

    normalize <- function(x, na.rm = TRUE) {
        ranx <- range(x, na.rm = na.rm)
        (x - ranx[1]) / diff(ranx)
    }
    
    x_exp.norm = apply(x_exp, 2, normalize)

    #--------------------------------------#
    # Kriging-based Sobol
    #--------------------------------------#
    res <- sobolGP_poro_np(
    model = "GP",
    type="UK",
    MCmethod="sobol2002",
    X1,
    X2,
    nsim = 50,
    conf = 0.95,
    nboot=1,
    sequential = FALSE,
    candidate,
    sequential.tot=FALSE,
    max_iter = 150,
    filename="Np_GP_loss_poro.py",
    nobs = i
    )

    # combine number of observation vector with Sobol indices
    data <- cbind(i, res$S$mean, t(res$S$ci[1,]), t(res$S$ci[2,]))
    dat <- as.data.frame(data)

    # total index
    dataT <- cbind(i, res$T$mean, t(res$T$ci[1,]), t(res$T$ci[2,]))
    datT <- as.data.frame(dataT)

    # write to file
    results_dir <- "../results/"
    filename <- paste(results_dir, paste("Np_GP_loss_poro_first",".csv", sep=""), sep = "")
    write.table(dat, filename, sep = ",", col.names = !file.exists(filename), row.names = F, append = T)

    filenameT <- paste(results_dir, paste("Np_GP_loss_poro_tot",".csv", sep=""), sep = "")
    write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), row.names = F, append = T)
}

In [34]:
res$S

0.2024666,0.7050228
0.2383298,0.3102915
-0.3031855,0.1109655
0.6008208,1.0840103
0.2383298,0.3102915
NA,NA
